In [30]:

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import spacy
# from utils import translate_sentence, bleu, save_checkpoint, load_checkpoint
# from torch.utils.tensorboard import SummaryWriter
# from torchtext.datasets import Multi30k
# from torchtext.data import Field, BucketIterator
from torch.nn import TransformerEncoder, TransformerEncoderLayer


In [33]:
from torch.utils.tensorboard import SummaryWriter
from torchtext.datasets import Multi30k
from torchtext.legacy import data
from torchtext.legacy.data import Field, BucketIterator

In [36]:
import numpy
print(numpy.__version__)

1.21.2


In [21]:
import sys
sys.path.insert(0, "/Users/scarr/MIMS/tensor-hero/Shared_Functionality/Preprocessing/Preprocessing Functions/")
from audio_functions import compute_mel_spectrogram

ImportError: Numba needs NumPy 1.20 or less

In [ ]:
sys.path.insert(0, "/Users/scarr/MIMS/tensor-hero/Model_1/")
from model_1_helper_functions_samantha import simplify_ohc_notes, create_matrix, make_time_and_note_array,create_pairs
song_path = "/Users/scarr/MIMS/files_tensorhero/Training Data/Unprocessed/Facelift Pack 2/Affiance - Call to the Warrior [Chezy]/song.ogg"


In [18]:
spec = compute_mel_spectrogram(song_path)
train_data = np.array(spec[:,300], spec[:,301], spec[:,302].spec[:,303], spec[:,304])
valid_data = np.array(spec[:,10000], spec[:,10001], spec[:,10002].spec[:,10003], spec[:,10004])

NameError: name 'compute_mel_spectrogram' is not defined

In [19]:
class Transformer(nn.Module):
    def __init__(
        self,
        embedding_size,
        # src_vocab_size,
        src_size,
        trg_vocab_size,
        # src_pad_idx,
        num_heads,
        num_encoder_layers,
        num_decoder_layers,
        forward_expansion,
        dropout,
        max_len,
        device,
    ):
        super(Transformer, self).__init__()
        #self.src_notes_embedding = nn.Embedding(src_vocab_size, embedding_size)
        self.src_notes_embedding = nn.Linear(src_size,embedding_size)
        self.src_position_embedding = nn.Embedding(max_len, embedding_size)
        self.trg_notes_embedding = nn.Embedding(trg_vocab_size, embedding_size)
        self.trg_position_embedding = nn.Embedding(max_len, embedding_size)

        self.device = device
        self.transformer = nn.Transformer(
            embedding_size,
            num_heads,
            num_encoder_layers,
            num_decoder_layers,
            forward_expansion,
            dropout,
        )
        self.fc_out = nn.Linear(embedding_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)
        # self.src_pad_idx = src_pad_idx

    # def make_src_mask(self, src):
    #     src_mask = src.transpose(0, 1) == self.src_pad_idx

    #     # (N, src_len)
    #     return src_mask.to(self.device)

    def forward(self, src, trg):
        src_seq_length, N = src.shape
        trg_seq_length, N = trg.shape

        src_positions = (
            torch.arange(0, src_seq_length)
            .unsqueeze(1)
            .expand(src_seq_length, N)
            .to(self.device)
        )

        trg_positions = (
            torch.arange(0, trg_seq_length)
            .unsqueeze(1)
            .expand(trg_seq_length, N)
            .to(self.device)
        )

        embed_src = self.dropout(
            (self.src_notes_embedding(src) + self.src_position_embedding(src_positions))
        )
        embed_trg = self.dropout(
            (self.trg_notes_embedding(trg) + self.trg_position_embedding(trg_positions))
        )

        # src_padding_mask = self.make_src_mask(src)
        # trg_mask = self.transformer.generate_square_subsequent_mask(trg_seq_length).to(
        #     self.device
        # )

        out = self.transformer(
            embed_src,
            embed_trg
            # src_key_padding_mask=src_padding_mask,
            # tgt_mask=trg_mask
        )
        out = self.fc_out(out)
        return out

In [14]:
# We're ready to define everything we need for training our Seq2Seq model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

load_model = True
save_model = False

# Training hyperparameters
num_epochs = 50
learning_rate = 3e-4
batch_size = 32

In [15]:
# Model hyperparameters
# src_vocab_size = 512
src_size = 512
trg_vocab_size = 31
embedding_size = 512
num_heads = 8
num_encoder_layers = 3
num_decoder_layers = 3
dropout = 0.10
max_len = 100
forward_expansion = 4

# src_pad_idx = english.vocab.stoi["<pad>"]

In [16]:
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=batch_size,
    sort_within_batch=True,
    sort_key=lambda x: len(x.src),
    device=device,
)

model = Transformer(
    embedding_size,
    src_vocab_size,
    trg_vocab_size,
    # src_pad_idx,
    num_heads,
    num_encoder_layers,
    num_decoder_layers,
    forward_expansion,
    dropout,
    max_len,
    device,
).to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

NameError: name 'train_data' is not defined

In [ ]:
# pad_idx = english.vocab.stoi["<pad>"]
# criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

criterion = nn.CrossEntropyLoss()

sentence = 

for epoch in range(num_epochs):
    print(f"[Epoch {epoch} / {num_epochs}]")

    model.eval()
    translated_sentence = translate_sentence(
        model, sentence, german, english, device, max_length=50
    )

    print(f"Translated example sentence: \n {translated_sentence}")
    model.train()
    losses = []

    for batch_idx, batch in enumerate(train_iterator):
        # Get input and targets and get to cuda
        inp_data = batch.src.to(device)
        target = batch.trg.to(device)

        # Forward prop
        output = model(inp_data, target[:-1, :])

        output = output.reshape(-1, output.shape[2])
        target = target[1:].reshape(-1)

        optimizer.zero_grad()

        loss = criterion(output, target)
        losses.append(loss.item())

        # Back prop
        loss.backward()
        # Clip to avoid exploding gradient issues, makes sure grads are within a healthy range
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

        # Gradient descent step
        optimizer.step()



    mean_loss = sum(losses) / len(losses)
